# Create combined subcortical atlas

It can be hard to know exactly where atlases came from and how they were processed after being downloaded.
This notebook downloads and merges some popular/very nice subcortical atlases for connectivity analysis.

In [1]:
%%bash

set -eux

for atlasname in CIT168 cerebellum Schaefer thalamus hippocampus_and_amygdala
do
  cd ${atlasname}
  bash -ex prepare_atlas.sh
  cd ..
done

+ for atlasname in CIT168 cerebellum Schaefer thalamus hippocampus_and_amygdala
+ cd CIT168
+ bash -ex prepare_atlas.sh
+ wget https://osf.io/download/6qrcb
--2023-10-30 20:40:54--  https://osf.io/download/6qrcb
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 308 PERMANENT REDIRECT
Location: https://osf.io/download/6qrcb/ [following]
--2023-10-30 20:40:54--  https://osf.io/download/6qrcb/
Reusing existing connection to osf.io:443.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/jkzwp/providers/osfstorage/5b11f996f1f288000f627962 [following]
--2023-10-30 20:40:54--  https://files.osf.io/v1/resources/jkzwp/providers/osfstorage/5b11f996f1f288000f627962
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https:

Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4006:822::201b, 2607:f8b0:4006:823::201b, 2607:f8b0:4006:80e::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4006:822::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51026 (50K) [application/octet-stream]
Saving to: ‘vak6p’

     0K .......... .......... .......... .......... ......... 100% 3.19M=0.02s

2023-10-30 20:41:00 (3.19 MB/s) - ‘vak6p’ saved [51026/51026]

+ mv vak6p tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-RAS_dseg.nii.gz
+ python split_up_cit168.py tpl-MNI152NLin6Asym_atlas-CIT168_res-01_desc-RAS_dseg.nii.gz tpl-MNI152NLin6Asym_atlas-CIT168_res-01_desc-LRSplit_dseg
+ python split_up_cit168.py tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-RAS_dseg.nii.gz tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-LRSplit_dseg
+ cp tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-LRSplit_dseg.tsv atl-CIT168.tsv
+ cd ..
+ for atlasname in CIT

Using double precision for computations.
Input scalar image: atl-MDTB10_space-MNI_dseg.nii
Reference image: /Users/mcieslak/projects/templateflow/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-01_desc-brain_mask.nii.gz
The composite transform comprises the following transforms (in order): 
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: tpl-MNI152NLin6Asym_atlas-MDTB10_res-01_dseg.nii.gz


+ antsApplyTransforms -d 3 -i atl-MDTB10_space-MNI_dseg.nii -o tpl-MNI152NLin2009cAsym_atlas-MDTB10_res-01_dseg.nii.gz -t /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5 -r /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz --interpolation GenericLabel -v 1


Using double precision for computations.
Input scalar image: atl-MDTB10_space-MNI_dseg.nii
Reference image: /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: tpl-MNI152NLin2009cAsym_atlas-MDTB10_res-01_dseg.nii.gz


...

+ python make_labels_tsv.py
+ cd ..
+ for atlasname in CIT168 cerebellum Schaefer thalamus hippocampus_and_amygdala
+ cd hippocampus_and_amygdala
+ bash -ex prepare_atlas.sh
+ wget https://raw.githubusercontent.com/PennLINC/xcp_d/8a07b53b8a1bb921cd8ca3dc2d23fd09da7982fd/xcp_d/data/atlases/atlas-HCP_dseg.tsv
--2023-10-30 20:46:30--  https://raw.githubusercontent.com/PennLINC/xcp_d/8a07b53b8a1bb921cd8ca3dc2d23fd09da7982fd/xcp_d/data/atlases/atlas-HCP_dseg.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027 (1.0K) [text/plain]
Saving to: ‘atlas-HCP_dseg.tsv’

     0K .                                                     100% 9.99M=0s

2023-10-30 20:46:30 (9.99 MB/s) - ‘atlas-HCP_dseg.tsv’ saved [1027/1027]

+ wget https://github.com/Washing

Using double precision for computations.
Input scalar image: tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-06_dseg.nii.gz
Reference image: /Users/mcieslak/projects/templateflow/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-01_desc-brain_mask.nii.gz
The composite transform comprises the following transforms (in order): 
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-01_dseg.nii.gz


+ antsApplyTransforms -d 3 -i tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-06_dseg.nii.gz -o tpl-MNI152NLin2009cAsym_atlas-HPandAMYG_res-01_dseg.nii.gz -t /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5 -r /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz --interpolation GenericLabel -v 1


Using double precision for computations.
Input scalar image: tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-06_dseg.nii.gz
Reference image: /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: tpl-MNI152NLin2009cAsym_atlas-HPandAMYG_res-01_dseg.nii.gz


+ cd ..


This has downloaded and standardized the atlases

In [2]:
import os
import nibabel as nb

tsvs = {
    "cerebellum": "cerebellum/atl-MDTB10.tsv",
    "subcortex": "CIT168/atl-CIT168.tsv",
    "thalamus": "thalamus/atlas-hcpthalamus.tsv",
    "hippocampus_and_amygdala": "hippocampus_and_amygdala/atlas-HPandAMYG_dseg.tsv",
}

tfhome = os.getenv("TEMPLATEFLOW_HOME")

nlin6_ref = os.path.join(
    tfhome,
    "tpl-MNI152NLin6Asym",
    "tpl-MNI152NLin6Asym_res-01_desc-brain_mask.nii.gz",
)
nlin6_ref_img = nb.load(nlin6_ref)
nlin6 = {
    "cerebellum": "cerebellum/tpl-MNI152NLin6Asym_atlas-MDTB10_res-01_dseg.nii.gz",
    "subcortex": "CIT168/tpl-MNI152NLin6Asym_atlas-CIT168_res-01_desc-LRSplit_dseg.nii.gz",
    "thalamus": "thalamus/tpl-MNI152NLin6Asym_atlas-hcpthalamic_res-01_dseg.nii.gz",
    "hippocampus_and_amygdala": (
        "hippocampus_and_amygdala/tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-01_dseg.nii.gz"
    ),
}

nlin09c_ref = os.path.join(
    tfhome,
    "tpl-MNI152NLin2009cAsym",
    "tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz",
)
nlin09c_ref_img = nb.load(nlin09c_ref)
nlin09c = {
    "cerebellum": "cerebellum/tpl-MNI152NLin2009cAsym_atlas-MDTB10_res-01_dseg.nii.gz",
    "subcortex": "CIT168/tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-LRSplit_dseg.nii.gz",
    "thalamus": "thalamus/tpl-MNI152NLin2009cAsym_atlas-hcpthalamic_res-01_dseg.nii.gz",
    "hippocampus_and_amygdala": (
        "hippocampus_and_amygdala/tpl-MNI152NLin2009cAsym_atlas-HPandAMYG_res-01_dseg.nii.gz"
    ),
}

In [3]:
from utils import check_grids

check_grids([nlin6_ref] + list(nlin6.values()))
check_grids([nlin09c_ref] + list(nlin09c.values()))

checking files:

  /Users/mcieslak/projects/templateflow/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-01_desc-brain_mask.nii.gz
  cerebellum/tpl-MNI152NLin6Asym_atlas-MDTB10_res-01_dseg.nii.gz
  CIT168/tpl-MNI152NLin6Asym_atlas-CIT168_res-01_desc-LRSplit_dseg.nii.gz
  thalamus/tpl-MNI152NLin6Asym_atlas-hcpthalamic_res-01_dseg.nii.gz
  hippocampus_and_amygdala/tpl-MNI152NLin6Asym_atlas-HPandAMYG_res-01_dseg.nii.gz

comparing grids:
(182, 218, 182) == (182, 218, 182)
(182, 218, 182) == (182, 218, 182)
(182, 218, 182) == (182, 218, 182)
(182, 218, 182) == (182, 218, 182)
All grids match orientation and shape!!
checking files:

  /Users/mcieslak/projects/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_mask.nii.gz
  cerebellum/tpl-MNI152NLin2009cAsym_atlas-MDTB10_res-01_dseg.nii.gz
  CIT168/tpl-MNI152NLin2009cAsym_atlas-CIT168_res-01_desc-LRSplit_dseg.nii.gz
  thalamus/tpl-MNI152NLin2009cAsym_atlas-hcpthalamic_res-01_dseg.nii.gz
  hippocampus_and_amygdala/tpl-M

Let's look at what one of these entries looks like:

In [4]:
from utils import roi_data, tsv_to_config, verify_atlas

for atlas in ["cerebellum", "thalamus", "subcortex", "hippocampus_and_amygdala"]:
    print(atlas)
    # Check nlin6
    verify_atlas(tsv_to_config(tsvs[atlas]), roi_data(nlin6[atlas]))

    # Check 2009c
    verify_atlas(tsv_to_config(tsvs[atlas]), roi_data(nlin09c[atlas]))

cerebellum
thalamus
subcortex
hippocampus_and_amygdala


The TSVs and the niftis all check out.

## Combining the atlases

We need to add the cerebellum, thalamus and subcortical regions into a single atlas. Below are some functions that help us add one atlas into another. We'll add these three together and then add them to all the other atlases that don't have any subcortical regions

Testing this code, let's use the cit168 atlas as the base and make sure these functions do what we think they do

In [5]:
import numpy as np

from utils import get_region_data, match_partitions, remap_values

# This simulates re-numbering the CIT168 altas
cit_labels, cit_names = get_region_data(tsvs["subcortex"])
new_atlas_mapping = {
    old_value: new_value
    for old_value, new_value in zip(cit_labels, np.argsort(cit_labels) + 1)
}

print("atlas mapping:")
print(*sorted(new_atlas_mapping.items()), sep="  ")

hemi_cit_data = nb.load(nlin6["subcortex"]).get_fdata().astype(np.uint32)
remapped_atlas = remap_values(hemi_cit_data, new_atlas_mapping)
print("matched labels:")
print(*sorted(match_partitions(hemi_cit_data, remapped_atlas).items()), sep="  ")

atlas mapping:
(1, 1)  (2, 2)  (3, 3)  (4, 4)  (5, 5)  (6, 6)  (7, 7)  (8, 8)  (9, 9)  (12, 10)  (13, 11)  (14, 12)  (15, 13)  (16, 14)  (101, 15)  (102, 16)  (103, 17)  (104, 18)  (105, 19)  (106, 20)  (107, 21)  (108, 22)  (109, 23)  (112, 24)  (113, 25)  (114, 26)  (115, 27)  (116, 28)
matched labels:
(1, array([1], dtype=uint32))  (2, array([2], dtype=uint32))  (3, array([3], dtype=uint32))  (4, array([4], dtype=uint32))  (5, array([5], dtype=uint32))  (6, array([6], dtype=uint32))  (7, array([7], dtype=uint32))  (8, array([8], dtype=uint32))  (9, array([9], dtype=uint32))  (12, array([10], dtype=uint32))  (13, array([11], dtype=uint32))  (14, array([12], dtype=uint32))  (15, array([13], dtype=uint32))  (16, array([14], dtype=uint32))  (101, array([15], dtype=uint32))  (102, array([16], dtype=uint32))  (103, array([17], dtype=uint32))  (104, array([18], dtype=uint32))  (105, array([19], dtype=uint32))  (106, array([20], dtype=uint32))  (107, array([21], dtype=uint32))  (108, array(

It works!! Now we can start adding these atlases to one another.



### Creating the full subcortical atlas

Here we merge the three subcortical atlases

In [8]:
import pandas as pd

from utils import add_atlas_to_another, expand_df
os.makedirs("subcortical_merged", exist_ok=True)
cit_hemi_config = tsv_to_config(tsvs["subcortex"])
cit_hemi_data_nlin6 = roi_data(nlin6["subcortex"])
cit_hemi_data_09c = roi_data(nlin09c["subcortex"])

thalamus_config = tsv_to_config(tsvs["thalamus"])
thalamus_config["node_names"] = [
    f"ThalamusHCP_{name}" for name in thalamus_config["node_names"]
]
thalamus_data_nlin6 = roi_data(nlin6["thalamus"])
thalamus_data_09c = roi_data(nlin09c["thalamus"])

cb_config = tsv_to_config(tsvs["cerebellum"])
cb_config["node_names"] = [
    f"Cerebellum_Cerebellar_{name}" for name in cb_config["node_names"]
]
cb_data_nlin6 = roi_data(nlin6["cerebellum"])
cb_data_09c = roi_data(nlin09c["cerebellum"])

hp_config = tsv_to_config(tsvs["hippocampus_and_amygdala"])
hp_config["node_names"] = [
    f"SubcorticalHCP_{name}" for name in hp_config["node_names"]
]
hp_data_nlin6 = roi_data(nlin6["hippocampus_and_amygdala"])
hp_data_09c = roi_data(nlin09c["hippocampus_and_amygdala"])

# Do NLin6
thalamus_and_cit_config_nlin6, thalamus_and_cit_data_nlin6 = add_atlas_to_another(
    cit_hemi_config,
    cit_hemi_data_nlin6,
    thalamus_config,
    thalamus_data_nlin6,
)

hp_thalamus_and_cit_config_nlin6, hp_thalamus_and_cit_data_nlin6 = add_atlas_to_another(
    thalamus_and_cit_config_nlin6,
    thalamus_and_cit_data_nlin6,
    hp_config,
    hp_data_nlin6,
)

full_subcortical_config_nlin6, full_subcortical_data_nlin6 = add_atlas_to_another(
    hp_thalamus_and_cit_config_nlin6,
    hp_thalamus_and_cit_data_nlin6,
    cb_config,
    cb_data_nlin6,
)

subcortical_nlin6 = nb.Nifti1Image(
    full_subcortical_data_nlin6,
    nlin6_ref_img.affine,
    header=nlin6_ref_img.header,
)
subcortical_nlin6.to_filename(
    "subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
)
nlin6_df = pd.DataFrame(
    {
        "index": full_subcortical_config_nlin6["node_ids"],
        "name": full_subcortical_config_nlin6["node_names"],
    }
)
expand_df(nlin6_df).to_csv(
    "subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.tsv",
    sep="\t",
    index=False,
)

In [9]:
# Do 09c
thalamus_and_cit_config_09c, thalamus_and_cit_data_09c = add_atlas_to_another(
    cit_hemi_config,
    cit_hemi_data_09c,
    thalamus_config,
    thalamus_data_09c,
)

hp_thalamus_and_cit_config_09c, hp_thalamus_and_cit_data_09c = add_atlas_to_another(
    thalamus_and_cit_config_09c,
    thalamus_and_cit_data_09c,
    hp_config,
    hp_data_09c,
)

full_subcortical_config_09c, full_subcortical_data_09c = add_atlas_to_another(
    hp_thalamus_and_cit_config_09c,
    hp_thalamus_and_cit_data_09c,
    cb_config,
    cb_data_09c,
)

subcortical_09c = nb.Nifti1Image(
    full_subcortical_data_09c,
    nlin09c_ref_img.affine,
    header=nlin09c_ref_img.header,
)
subcortical_09c.to_filename(
    "subcortical_merged/tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
)
nlin09c_df = pd.DataFrame(
    {
        "index": full_subcortical_config_09c["node_ids"],
        "name": full_subcortical_config_09c["node_names"],
    }
)
expand_df(nlin09c_df).to_csv(
    "subcortical_merged/tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.tsv",
    sep="\t",
    index=False,
)

## Downsample combined subcortical atlas to 2 mm3 for CIFTI version

In [14]:
%%bash

antsApplyTransforms \
    -d 3 \
    -i 'subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz' \
    -o 'subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_dseg.nii.gz' \
    -r ${TEMPLATEFLOW_HOME}'/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz' \
    --interpolation GenericLabel \
    -v 1


Using double precision for computations.
Input scalar image: subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz
Reference image: /Users/mcieslak/projects/templateflow/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz
The composite transform comprises the following transforms (in order): 
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: subcortical_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_dseg.nii.gz


# Generate CIFTI files

In [16]:
%%bash

cd Schaefer/
python 01_prepare_cifti_atlas.py
bash 02_generate_subcortical_cifti.sh
cd ..

100%|██████████| 3.54k/3.54k [00:00<00:00, 882kB/s]
100%|██████████| 3.51k/3.51k [00:00<00:00, 1.36MB/s]
Traceback (most recent call last):
  File "01_prepare_cifti_atlas.py", line 162, in <module>
    download_templates()
  File "01_prepare_cifti_atlas.py", line 89, in download_templates
    shutil.copyfile(rh, os.path.basename(rh))
  File "/usr/local/Caskroom/miniconda/base/envs/qsiprep/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: 'tpl-fsLR_hemi-R_den-32k_desc-nomedialwall_dparc.label.gii'


100



While running:
/Applications/workbench/bin_macosx64/../macosx64_apps/wb_command.app/Contents/MacOS/wb_command -cifti-separate tpl-fsLR_atlas-Schaefer2018v0143_den-32k_desc-100ParcelsAllNetworks_dseg.dlabel.nii COLUMN -label CORTEX_LEFT tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-L_desc-100ParcelsAllNetworks_dseg.label.gii -label CORTEX_RIGHT tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-R_desc-100ParcelsAllNetworks_dseg.label.gii

ERROR: failed to open file '/Users/mcieslak/projects/AtlasPack/Schaefer/tpl-fsLR_atlas-Schaefer2018v0143_den-32k_desc-100ParcelsAllNetworks_dseg.dlabel.nii', file does not exist, or folder permissions prevent seeing it



    Converting NIFTI atlas to NIFTI label file.



While running:
/Applications/workbench/bin_macosx64/../macosx64_apps/wb_command.app/Contents/MacOS/wb_command -volume-label-import tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x100Parcels_dseg.nii.gz tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x100Parcels_labellist.txt tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x100ParcelsLabels_dseg.nii.gz

ERROR: NAME OF FILE: tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x100Parcels_dseg.nii.gz

File does not exist.



...


While running:
/Applications/workbench/bin_macosx64/../macosx64_apps/wb_command.app/Contents/MacOS/wb_command -cifti-separate tpl-fsLR_atlas-Schaefer2018v0143_den-32k_desc-1000ParcelsAllNetworks_dseg.dlabel.nii COLUMN -label CORTEX_LEFT tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-L_desc-1000ParcelsAllNetworks_dseg.label.gii -label CORTEX_RIGHT tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-R_desc-1000ParcelsAllNetworks_dseg.label.gii

ERROR: failed to open file '/Users/mcieslak/projects/AtlasPack/Schaefer/tpl-fsLR_atlas-Schaefer2018v0143_den-32k_desc-1000ParcelsAllNetworks_dseg.dlabel.nii', file does not exist, or folder permissions prevent seeing it



    Converting NIFTI atlas to NIFTI label file.



While running:
/Applications/workbench/bin_macosx64/../macosx64_apps/wb_command.app/Contents/MacOS/wb_command -volume-label-import tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000Parcels_dseg.nii.gz tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000Parcels_labellist.txt tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000ParcelsLabels_dseg.nii.gz

ERROR: NAME OF FILE: tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000Parcels_dseg.nii.gz

File does not exist.



    Converting NIFTI label file to CIFTI



While running:
/Applications/workbench/bin_macosx64/../macosx64_apps/wb_command.app/Contents/MacOS/wb_command -cifti-create-label ../tpl-fsLR_atlas-4S1056Parcels_den-91k_dseg.dlabel.nii -volume tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000ParcelsLabels_dseg.nii.gz Atlas_ROIs.2.nii.gz -left-label tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-L_desc-1000ParcelsAllNetworks_dseg.label.gii -roi-left tpl-fsLR_hemi-L_den-32k_desc-nomedialwall_dparc.label.gii -right-label tpl-fsLR_atlas-Schaefer2018v0143_den-32k_hemi-R_desc-1000ParcelsAllNetworks_dseg.label.gii -roi-right tpl-fsLR_hemi-R_den-32k_desc-nomedialwall_dparc.label.gii

ERROR: NAME OF FILE: tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-02_desc-Schaefer2018v0143x1000ParcelsLabels_dseg.nii.gz

File does not exist.



# Generate metadata files

In [17]:
%%bash

cd metadata/
python generate_metadata.py
cd ..

  0%|          | 0/10 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "generate_metadata.py", line 353, in <module>
    combine_metadata(n_parcels)
  File "generate_metadata.py", line 298, in combine_metadata
    with open(merged_nifti_metadata_file, "w") as fo:
FileNotFoundError: [Errno 2] No such file or directory: '../4S/tpl-MNI152NLin6Asym_atlas-4S152Parcels_dseg.json'
